# Import

In [74]:
import numpy as np
import pandas as pd
import datetime

import sys, os
sys.path.insert(0, os.path.abspath('scripts'))
import rus_table_parser

# Data parsing

In [75]:
DAY = datetime.datetime.now().strftime("%d").lstrip('0')
MONTH = datetime.datetime.now().strftime("%m").lstrip('0')
# DAY = 25
# MONTH = 5

# Save of "https://coronavirus-monitor.ru/coronavirus-v-rossii/" page
path = f'html_data/{DAY}-{MONTH}/Коронавирус в России. Онлайн карта распространения коронавируса в России..html'
parse_df = rus_table_parser.run_parsing(path)
parse_df

,Confirmed,Date,Deaths,Recovered,Region/City
0,180791.0,2020-05-31,2477.0,80179.0,Москва
1,38995.0,2020-05-31,449.0,8091.0,Московская область
2,15949.0,2020-05-31,204.0,5680.0,Санкт-Петербург
3,9834.0,2020-05-31,98.0,3693.0,Нижегородская область
4,5449.0,2020-05-31,27.0,2173.0,Свердловская область
...,...,...,...,...,...
80,161.0,2020-05-31,3.0,131.0,Севастополь
81,122.0,2020-05-31,0.0,71.0,Республика Алтай
82,115.0,2020-05-31,0.0,53.0,Сахалинская область
83,80.0,2020-05-31,1.0,54.0,Чукотский автономный округ


# Preparing parsed data

In [76]:
file_name = 'release/covid19-russia-cases.csv'
rus_df = pd.read_csv(file_name)
rus_df.tail()

,Date,Region/City,Region/City-Eng,Region_ID,Day-Confirmed,Day-Deaths,Day-Recovered,Confirmed,Deaths,Recovered
4833,2020-05-30,Челябинская область,Chelyabinsk region,74.0,292.0,2.0,152.0,2878.0,17.0,1258.0
4834,2020-05-30,Чеченская Республика,Chechen Republic,95.0,17.0,0.0,38.0,1209.0,13.0,825.0
4835,2020-05-30,Чукотский автономный округ,Chukotka Autonomous Okrug,87.0,5.0,0.0,3.0,78.0,1.0,53.0
4836,2020-05-30,Ямало-Ненецкий АО,Yamalo-Nenets Autonomous Okrug,89.0,30.0,0.0,55.0,2310.0,9.0,953.0
4837,2020-05-30,Ярославская область,Yaroslavl region,76.0,79.0,0.0,53.0,3055.0,14.0,751.0


In [77]:
# Create day-columns
parse_df['Day-Confirmed'] = 0
parse_df['Day-Deaths'] = 0
parse_df['Day-Recovered'] = 0

In [78]:
# Strip text data
rus_df['Region/City'] = rus_df['Region/City'].astype('str').str.strip('\u200b')
parse_df['Region/City'] = parse_df['Region/City'].astype('str').str.strip('\u200b')

In [79]:
# Rename regions
rename_dict = {
    'Татарстан' : 'Республика Татарстан',
    'Башкортостан' : 'Республика Башкортостан',
    'Чувашская Республика': 'Республика Чувашия',
    'Камчатский край' : 'Камчатский край',
    'Ямало-Ненецкий автономный округ' : 'Ямало-Ненецкий АО',
    'Республика Северная Осетия — Алания' : 'Республика Северная Осетия - Алания',
    'Республика Алтай' : 'Республика Алтай'
}

parse_df['Region/City'] = parse_df['Region/City'].replace(rename_dict)

In [80]:
parse_df['Date'] = pd.to_datetime(parse_df['Date'])
# Minus a day from now if needed
# parse_df['Date'] = parse_df['Date'] - pd.Timedelta(days=1)

In [81]:
# Fit day-columns
def upd(row):
    reg = row['Region/City']
    
    row['Day-Confirmed'] = row['Confirmed'] - rus_df[rus_df['Region/City'] == reg]['Confirmed'].max()
    row['Day-Deaths']    = row['Deaths'] - rus_df[rus_df['Region/City'] == reg]['Deaths'].max()
    row['Day-Recovered'] = row['Recovered'] - rus_df[rus_df['Region/City'] == reg]['Recovered'].max()
    
    row['Day-Confirmed'] = row['Confirmed'] if np.isnan(row['Day-Confirmed']) else row['Day-Confirmed']
    row['Day-Deaths']    = row['Deaths'] if np.isnan(row['Day-Deaths']) else row['Day-Deaths']
    row['Day-Recovered'] = row['Recovered'] if np.isnan(row['Day-Recovered']) else row['Day-Recovered']
    
    return row.drop('Region/City')

parse_df = parse_df.groupby('Region/City').apply(lambda df: upd(df.iloc[0])).reset_index()

In [82]:
# Check for missed regions
parse_df[np.isnan(parse_df['Day-Confirmed'])]

,Region/City,Confirmed,Date,Deaths,Recovered,Day-Confirmed,Day-Deaths,Day-Recovered


# Data checking

In [83]:
print('Russia Confirmed:', parse_df.groupby('Region/City')['Confirmed'].max().sum(),
      'Day-Confirmed:', parse_df.groupby('Region/City')['Day-Confirmed'].sum().sum())
print('Russia Deaths:', parse_df.groupby('Region/City')['Deaths'].max().sum(),
      'Day-Deaths:', parse_df.groupby('Region/City')['Day-Deaths'].sum().sum())
print('Russia Recovered:', parse_df.groupby('Region/City')['Recovered'].max().sum(),
      'Day-Recovered:', parse_df.groupby('Region/City')['Day-Recovered'].sum().sum())

# rus_regs = rus_sum.groupby('Region/City')['Recovered'].max().reset_index()
# rus_regs['Recovered-ByDay'] = rus_sum.groupby('Region/City')['Day-Recovered'].sum().reset_index(drop=True)
# rus_regs.loc[rus_regs['Recovered'] != rus_regs['Recovered-ByDay']]
# # rus_regs.to_csv('rus_regs.csv')

Russia Confirmed: 405926.0 Day-Confirmed: 9013.0
Russia Deaths: 4698.0 Day-Deaths: 140.0
Russia Recovered: 172046.0 Day-Recovered: 4363.0


# Data saving

In [84]:
# Save full snapshot of current day
parse_df.to_csv(f'covid19-russia-cases-upd-full.csv', index=False)

In [85]:
# Filter regions without changes
parse_df = parse_df[(parse_df['Day-Confirmed'] != 0) |
                (parse_df['Day-Deaths'] != 0) |
                (parse_df['Day-Recovered'] != 0)]

In [86]:
# Save parsed data for future use
parse_df.to_csv(f'old_data/covid19-russia-cases-upd{DAY}-{MONTH}.csv', index=False)
parse_df.to_csv('covid19-russia-cases-upd.csv', index=False)